FINETUNING


In [10]:
!pip install accelerate -U

In [6]:
!pip install transformers[torch]

In [7]:
!pip show accelerate

Name: accelerate
Version: 0.32.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, ProgressCallback
import torch
from sklearn.metrics import accuracy_score, classification_report

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Load your dataset
df_test = pd.read_csv('test_hate_dataset.csv')
df_train = pd.read_csv('train_hate_dataset.csv')

# Create binary labels: 0 for non-hate speech (offensive and neither), 1 for hate speech
df_test['label'] = df_test['label'].apply(lambda x: 1 if x == 'hate' else 0)
df_train['label'] = df_train['label'].apply(lambda x: 1 if x == 'hate' else 0)

train_texts = df_train['text']
train_labels = df_train['label']
test_texts = df_test['text']
test_labels = df_test['label']

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-hate')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-hate').to(device)

# Tokenize the texts
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

# Create a Dataset class
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}  # Convert lists to tensors
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = HateSpeechDataset(train_encodings, train_labels.values) # Remove .to(device) here
test_dataset = HateSpeechDataset(test_encodings, test_labels.values) # Remove .to(device) here

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"      # evaluate at the end of each epoch
)

# Define the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    callbacks=[ProgressCallback()]       # list of callbacks to apply during training
)

# Train the model
trainer.train()



Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strat

  0%|          | 0/3086 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
1,0.389200,0.416626
2,0.247300,0.405995


{'loss': 0.9495, 'grad_norm': 17.97427749633789, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.8864, 'grad_norm': 6.453651428222656, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.7309, 'grad_norm': 13.393362045288086, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.5941, 'grad_norm': 5.61953592300415, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}
{'loss': 0.713, 'grad_norm': 7.293231010437012, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.6369, 'grad_norm': 7.0598673820495605, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6323, 'grad_norm': 6.552701950073242, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6226, 'grad_norm': 7.004067420959473, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.5819, 'grad_norm': 6.716711044311523, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.5833, 'grad_norm': 8.41675853729248, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.5389, 'grad_norm': 13.221358

  0%|          | 0/1029 [00:00<?, ?it/s]

{'eval_loss': 0.41662630438804626, 'eval_runtime': 528.0961, 'eval_samples_per_second': 31.165, 'eval_steps_per_second': 1.949, 'epoch': 1.0}
{'loss': 0.3846, 'grad_norm': 12.011846542358398, 'learning_rate': 2.9698375870069606e-05, 'epoch': 1.0}
{'loss': 0.2732, 'grad_norm': 5.617614269256592, 'learning_rate': 2.9505027068832174e-05, 'epoch': 1.01}
{'loss': 0.284, 'grad_norm': 9.397047996520996, 'learning_rate': 2.9311678267594746e-05, 'epoch': 1.02}
{'loss': 0.2556, 'grad_norm': 15.418947219848633, 'learning_rate': 2.9118329466357308e-05, 'epoch': 1.02}
{'loss': 0.2479, 'grad_norm': 8.415506362915039, 'learning_rate': 2.8924980665119876e-05, 'epoch': 1.03}
{'loss': 0.3241, 'grad_norm': 7.444894790649414, 'learning_rate': 2.8731631863882448e-05, 'epoch': 1.04}
{'loss': 0.3566, 'grad_norm': 13.300630569458008, 'learning_rate': 2.853828306264501e-05, 'epoch': 1.04}
{'loss': 0.4446, 'grad_norm': 6.887020587921143, 'learning_rate': 2.834493426140758e-05, 'epoch': 1.05}
{'loss': 0.3541, 'g

  0%|          | 0/1029 [00:00<?, ?it/s]

{'eval_loss': 0.4059951603412628, 'eval_runtime': 527.484, 'eval_samples_per_second': 31.201, 'eval_steps_per_second': 1.951, 'epoch': 2.0}
{'train_runtime': 5834.8492, 'train_samples_per_second': 8.462, 'train_steps_per_second': 0.529, 'train_loss': 0.3979966111254244, 'epoch': 2.0}


TrainOutput(global_step=3086, training_loss=0.3979966111254244, metrics={'train_runtime': 5834.8492, 'train_samples_per_second': 8.462, 'train_steps_per_second': 0.529, 'total_flos': 1.299031902523392e+16, 'train_loss': 0.3979966111254244, 'epoch': 2.0})

In [3]:
# Evaluate the model
predictions, labels, _ = trainer.predict(test_dataset)
predictions = torch.argmax(torch.tensor(predictions), dim=1)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions.cpu().numpy())
print(f'Accuracy: {accuracy:.4f}')

# Print detailed classification report
print(classification_report(test_labels, predictions.cpu().numpy(), target_names=['Non-Hate Speech', 'Hate Speech']))


  0%|          | 0/1029 [00:00<?, ?it/s]

Accuracy: 0.8392
                 precision    recall  f1-score   support

Non-Hate Speech       0.82      0.83      0.83      7625
    Hate Speech       0.85      0.85      0.85      8833

       accuracy                           0.84     16458
      macro avg       0.84      0.84      0.84     16458
   weighted avg       0.84      0.84      0.84     16458



In [5]:
df = pd.read_csv('test_hate_dataset.csv')

print( df['label'].value_counts()['nothate'])
df['prediction'] = predictions
df.head()
df.to_csv('classified_sentences_finetuned_2.csv', index=False)

7625
